In [13]:
import pandas as pd
import numpy as np
import math
import operator 

In [66]:
# Example of kNN implemented from Scratch in Python

import csv
import random

def loadDataset(filename):
    df = pd.read_csv(filename, sep=';')
    df["quality"] = (df["quality"] > 5).astype(np.int)
    df.isnull().any()
    df = df.dropna()
    df = df.sample(frac=1)
    
    # Looking for numerical correlations between the attributes 
    # and target
    df.corr().abs()['quality'].sort_values(ascending=False)
    
    # correlation: density vs others
    df.corr().abs()['density'].sort_values(ascending=False)
    
    # Because 'residual sugar' and 'alcohol' are highly correlated 
    # to density, let check there are related to each other
    df.corr().abs()['alcohol'].sort_values(ascending=False)
    
    # Because 'residual sugar' and 'alcohol' are not highly 
    # correlated, we drop only 'density'
    df = df.drop(['density' ], 1)
    print(df.shape)

In [59]:
testLoadData = False

if testLoadData:
    split = 0.67
    trainingSet = []
    testSet = []
    loadDataset('datasets/DataScienceRepository/winequality-white.csv',\
                split, trainingSet, testSet)

In [67]:
"""
This function takes 3 arguments: feature matrix (numpy array with rows representing data samples and columns representing 
features.), target vector (numpy array with labels corresponding to each row of the feature matrix), and t ( a real number
to determine the size of partition). 
"""
def partition(features, target, t):
#     # Check if 'features' or 'target' is an instance of 'np.ndarray'
#     features = np.zeros(len(np_features))
#     target = np.zeros(len(np_features))
#     if isinstance(np_features, np.ndarray):
#         features = pd.DataFrame(np_features, index=range(np_features.shape[0]),
#                           columns=range(np_features.shape[1]))
#     if isinstance(np_target, np.ndarray):
#         target = pd.DataFrame(np_target, index=range(np_target.shape[0]),
#                           columns=range(np_target.shape[1]))
        
        
#     rowIndex = int((features.shape[0]*t))
    
#     X_test = np.array( features.iloc[:rowIndex]   )
#     X_train = np.array( features.iloc[rowIndex:] )

#     target_test = np.array( target.iloc[:rowIndex]  )
#     target_train = np.array( target.iloc[rowIndex:] )

    # create f and t; type np
    feat = np.zeros(target.shape[0])
    tar = np.zeros(target.shape[0])
    #cast DataFrame to np.array
    feat = np.array(features)
    tar = np.array(target)
    
    index = int(target.shape[0]*t)

    X_test = feat[:index,:]
    X_train = feat[index:, :]
    
    y_test = tar[:index]
    y_train = tar[index:]
    
    return  X_train, X_test, y_train, y_test

In [72]:
testPartition = True

if testPartition:
    X_train,\
    X_test,\
    y_train,\
    y_test  = partition(df.drop(["quality"],\
                        axis=1),\
                        df["quality"], t =0.2)
    print(X_train.shape,\
          X_test.shape,\
          y_train.shape,\
          y_test.shape)

(3919, 11) (979, 11) (3919,) (979,)


In [7]:
def euclideanDistance(instance1, instance2, length):
	distance = 0
	for x in range(length):
		distance += pow((instance1[x] - instance2[x]), 2)
	return math.sqrt(distance)

In [71]:
testDistance = True
if testDistance:
    data1 = [2, 2, 2, 'a']
    data2 = [4, 4, 4, 'b']
    #distance = euclideanDistance(data1, data2, 3)
    distance = euclideanDistance(X_train, X_test, len(X_train))
    print('Distance: ', distance)

IndexError: index 979 is out of bounds for axis 0 with size 979

In [14]:
def getNeighbors(trainingSet, testInstance, k):
	distances = []
	length = len(testInstance)-1
	for x in range(len(trainingSet)):
		dist = euclideanDistance(testInstance, trainingSet[x], length)
		distances.append((trainingSet[x], dist))
	distances.sort(key=operator.itemgetter(1))
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x][0])
	return neighbors

In [15]:
testNeighbors = True
if testNeighbors:
    trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
    testInstance = [5, 5, 5]
    k = 1
    neighbors = getNeighbors(trainSet, testInstance, 1)
    print(neighbors)

[[4, 4, 4, 'b']]


In [24]:
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(),\
                       key=operator.itemgetter(1),\
                       reverse=True)
    return sortedVotes[0][0]

In [28]:
testResponse = False

if testResponse:
    neighbors = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
    response = getResponse(neighbors)
    print(response)

In [26]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] is predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [29]:
testAccuracy = False
if testAccuracy:
    testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
    predictions = ['a', 'a', 'a']
    accuracy = getAccuracy(testSet, predictions)
    print(accuracy)

In [33]:
def main():
	# prepare data
	trainingSet=[]
	testSet=[]
	split = 0.67
	loadDataset(np.array(df), split, trainingSet, testSet)
	print('Train set: ' + repr(len(trainingSet)))
	print('Test set: ' + repr(len(testSet)))
	# generate predictions
	predictions=[]
	k = 3
	for x in range(len(testSet)):
		neighbors = getNeighbors(trainingSet, testSet[x], k)
		result = getResponse(neighbors)
		predictions.append(result)
		print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
	accuracy = getAccuracy(testSet, predictions)
	print('Accuracy: ' + repr(accuracy) + '%')
	
main()

TypeError: expected str, bytes or os.PathLike object, not numpy.ndarray